In [1]:
import pandas as pd
from torch.utils.data import DataLoader
import os
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.auto import tqdm
from PIL import Image
import torchvision.transforms as transforms
from torchvision import models
from torchvision.models import ResNet18_Weights

In [13]:
import pandas as pd
from torch.utils import data
import numpy as np

train_nrows = 28100
test_nrows = 7026
def getData(mode):
    if mode == 'train':
        img = pd.read_csv('train_img.csv', header=None, nrows=train_nrows)
        label = pd.read_csv('train_label.csv', header=None, nrows=train_nrows)
        return np.squeeze(img.values), np.squeeze(label.values)
    else:
        img = pd.read_csv('test_img.csv', header=None, nrows=test_nrows)
        label = pd.read_csv('test_label.csv', header=None, nrows=test_nrows)
        return np.squeeze(img.values), np.squeeze(label.values)

class RetinopathyLoader(data.Dataset):
    def __init__(self, root, mode):
        self.root = root
        self.img_name, self.label = getData(mode)
        self.mode = mode
        print("> Found %d images..." % (len(self.img_name)))

    def __len__(self):
        return len(self.img_name)

    def __getitem__(self, index):
        label = self.label[index]
        img_id = self.img_name[index]
        img_path = self.root+str(img_id)+".jpeg"
        img = np.array(Image.open(img_path))
        img = np.transpose(img,(2,0,1))
        img = img.astype('float32')
        return img, label

In [8]:
# model = models.resnet50()
# model = models.resnet50(ResNet50_Weights.IMAGENET1K_V2)
# model.fc = nn.Linear(2048, 5)

In [4]:
# model = models.resnet18()
model = models.resnet18(ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(512, 5)

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"
Epochs = 10
batch_size = 8
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, lr=0.0001, weight_decay=1e-5) 

In [16]:
train_set = RetinopathyLoader('./data/','train')
train_loader = DataLoader(train_set, batch_size=batch_size)

> Found 28100 images...


In [17]:
test_set = RetinopathyLoader('./data/','test')
test_loader = DataLoader(test_set, batch_size=batch_size)

> Found 7026 images...


## Training and Testing

In [70]:
# training and testing
total_trainp = []
total_testp = []
for epoch in (range(Epochs)):
    # training
    model.train()
    trainp = 0
    for img,lab in tqdm(train_loader):
        
        img, lab = img.to(device), lab.to(device)
        pred = model(img)
        loss = criterion(pred, lab)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        acc = (pred.argmax(dim=-1) == lab.to(device)).float().mean()
        trainp+=acc*len(lab)
    
    total_trainp.append(trainp/len(train_set))
    print(trainp/len(train_set))
    
    # testing
    model.eval()
    testp = 0
    for timg,tlab in tqdm(test_loader):
        
        timg, tlab = timg.to(device), tlab.to(device)
        with torch.no_grad():
            predd = model(timg)
        loss = criterion(predd,tlab)
        
        acc = (predd.argmax(dim=-1) == tlab.to(device)).float().mean()
        testp += acc*len(tlab)
        
    total_testp.append(testp/len(test_set))
    print(testp/len(test_set))

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

## Testing

In [20]:
model.load_state_dict(torch.load('80/18pre_80/ResNet18_pre_10s.pth'))

<All keys matched successfully>

In [21]:
model.eval()
testp = 0
for timg,tlab in tqdm(test_loader):
    timg, tlab = timg.to(device), tlab.to(device)
    with torch.no_grad():
        predd = model(timg)
    loss = criterion(predd,tlab)
    acc = (predd.argmax(dim=-1) == tlab.to(device)).float().mean()
    testp += acc*len(tlab)
print(testp/len(test_set))

  0%|          | 0/879 [00:00<?, ?it/s]

tensor(0.8012, device='cuda:0')


In [ ]:
# torch.save(model.state_dict(), 'ResNet18_10s.pth')

In [ ]:
# torch.save(total_trainp,'Resnet18_nopretrain.pt')

In [ ]:
# torch.save(total_testp,'Resnet18_nopretest.pt')